# Introduction to Remote Sensing with Python

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

- [Stewart Udall, Secretary of the Interior, September 21, 1966](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/1966.09.21-DOI-Udall-Earth%20Resources%20Studied%20From%20Space.pdf)

## A remote sensing crash course

<img src="images/remote.png" width=600>

https://youtu.be/DGE-N8_LQBo


[Incendio Vesuvio 2017](https://www.vesuviolive.it/ultime-notizie/256710-11-luglio-2017-a-fuoco-il-vesuvio-e-il-piu-grande-disastro-ambientale-della-sua-storia/)

[Sentinel 2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#description)

## Landsat

Landsat is the first "civilian Earth observation satellite" launched in 1972, a collaboration between the Department of the Interior, NASA, and the Department of Agriculture.

At over 40 years, the Landsat series of satellites provides the longest temporal record of moderate resolution multispectral data of the Earth’s surface on a global basis. The Landsat record has remained remarkably unbroken, proving a unique resource to assist a broad range of specialists in managing the world’s food, water, forests, and other natural resources for a growing world population. It is a record unmatched in quality, detail, coverage, and value. Source: USGS

- [NASA's Landsat page](https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- [USGS's Landsat page](https://www.usgs.gov/core-science-systems/nli/landsat)

## How to access landsat data

- [USGS Earth Explorer](https://earthexplorer.usgs.gov/)
- [Amazon AWS Open Data](https://registry.opendata.aws/landsat-8/)
- [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/landsat-8)

### The Workflow

In this lab, we will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest) in the Northern California's Butte County
- import multiple Landsat images from the dates before and after the Camp Fire
- determine which images are best for analysis
- create various NDVI map outputs to assess the amount of fire damage

## Python libraries in this workshop

- [pandas](https://pandas.pydata.org/docs/getting_started/index.html)
- [geopandas](https://geopandas.org/en/stable/gallery/index.html)
- [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)


In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import geopandas as gpd
import ee

from IPython.display import Image

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

For this lab, we will use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [ ]:
# coordinates of Vesuvius
lat = 40.821330
lon = 14.426102

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2017-05-10'

# end date
end_date = '2017-08-10'

## Get Sentinel 2 data

In [ ]:
# get the satellite data
sentinel = ee.ImageCollection(
    "COPERNICUS/S2_SR_HARMONIZED"
).filterBounds(poi).filterDate(start_date,end_date)

In [ ]:
# how many images did we get?
print('Total number:', sentinel.size().getInfo())

In [ ]:
# information about the first image in our collection
# sentinel.first().getInfo()

In [ ]:
# what about cloud cover of our first image?
# sentinel.first().get('CLOUD_COVERAGE_ASSESSMENT').getInfo()

In [ ]:
# what bands did we get?
# sentinel.first().bandNames().getInfo()

In [ ]:
# put the images in a list
sentinel_list = sentinel.toList(sentinel.size());

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine)

In [ ]:
# set some parameters for the images
parameters = {
                'min': 0,
                'max': 3_000,
                'dimensions': 1_000, # square size in pixels
                'bands': ['B4', 'B3', 'B2'] # bands to display (r,g,b)
             }

In [ ]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(sentinel.size().getInfo()):

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')
    
    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    
    # print the image info
    print('Image #',i,date_time,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date_time,cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Datetime', 'Cloud Cover'])

In [ ]:
df

## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [ ]:
# create a list of images we want (before, during, after)
sentinel_sequence = [0,9]

In [ ]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(6_000) # meters

In [ ]:
parameters = {
                'min': 0,
                'max': 3_000,
                'dimensions': 800,
                'bands': ['B4', 'B3', 'B2'],
                'region':roi
             }

In [ ]:
for i in sentinel_sequence:
    
    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    
    print('Image #',i,date_time,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

<div class="alert alert-info">
Take a moment to inspect the three images above. What do they tell you? What do they NOT tell you?
</div>

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

<img src="images/ndvi.png" width=600>

- [Source: Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/)

### Calculating NDVI in Google Earth Engine

- https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [ ]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 1024,
                   'palette': palette,
                   'region': roi}

In [ ]:
for i in sentinel_sequence:

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    print('Image #',i,date_time)
    
    # display the image
    display(
        Image(
            url=ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']).getThumbUrl(ndvi_parameters)
        )
    )

## Folium

Of course, we can't end the lab without an interactive map. For this, we can use [folium](https://python-visualization.github.io/folium/quickstart.html), which uses the open-source javascript mapping library [leaflet](https://leafletjs.com/).

- https://python-visualization.github.io/folium/quickstart.html

Google earth engine works with folium:
- https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

In [ ]:
! pip install folium --quiet

In [ ]:
import folium

# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=13)

# Add a layer for each satellite image of interest (before, during and after)
for i in sentinel_sequence:

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    my_map.add_ee_layer(
        ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']), 
        ndvi_parameters,
        name=date_time,
    )
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Save the folium map as an html file

In [ ]:
my_map.save('vesuvius.html')